In [214]:
using Enzyme

## Linear elastic rheology 2D
Two constant moduli: $K$ and $G$

$$
\tau = D \mathbf{\varepsilon}
$$

$$
\mathbf{\tau} = \begin{pmatrix} \tau_{xx} \\ \tau_{yy} \\ \tau_{xy} \\ P \end{pmatrix}
$$

$$
\mathbf{\varepsilon} = \begin{pmatrix} \varepsilon_{xx} \\ \varepsilon_{yy} \\ \varepsilon_{xy} \\ \nabla u \end{pmatrix}
$$

In [215]:
Stress_Elastic(ε, K, G) = [
    2*G*ε[1],
    2*G*ε[2],
    2*G*ε[3],
     -K*ε[4]
]

Stress_Elastic (generic function with 1 method)

In [216]:
ε = [-1e-4, 1e-4, 0., 1e-5]
K, G = 1e11, 5e10
J = Enzyme.jacobian(ReverseWithPrimal, Stress_Elastic, ε, K, G)
display(J.val)
display(J.derivs[1])

4-element Vector{Float64}:
 -1.0e7
  1.0e7
  0.0
 -1.0000000000000001e6

4×4 transpose(::Matrix{Float64}) with eltype Float64:
 1.0e11  0.0     0.0      0.0
 0.0     1.0e11  0.0      0.0
 0.0     0.0     1.0e11   0.0
 0.0     0.0     0.0     -1.0e11

## Linear viscous rheology 2D
A single modulus: $\eta$

$$\tau = D \mathbf{\dot{\varepsilon}}$$

$$
\mathbf{\tau} = \begin{pmatrix} \tau_{xx} \\ \tau_{yy} \\ \tau_{xy}\end{pmatrix}
$$

$$
\mathbf{\varepsilon} = \begin{pmatrix} \varepsilon_{xx} \\ \varepsilon_{yy} \\ \varepsilon_{xy} \end{pmatrix}
$$

In [217]:
Stress_Viscous(ε̇, η) = [
    2*η*ε̇[1],
    2*η*ε̇[2],
    2*η*ε̇[3]
]

Stress_Viscous (generic function with 1 method)

In [218]:
ε̇ = [-1e-15, 1e-15, 1e-16]
η = 1e22
J = Enzyme.jacobian(ReverseWithPrimal, Stress_Viscous, ε̇, η)
display(J.val)
display(J.derivs[1])

3-element Vector{Float64}:
 -2.0e7
  2.0e7
  2.0e6

3×3 transpose(::Matrix{Float64}) with eltype Float64:
 2.0e22  0.0     0.0
 0.0     2.0e22  0.0
 0.0     0.0     2.0e22

## Power-law viscous rheology 2D

Deviatoric strain rate invariant: $\dot{\varepsilon}_{II} = \sqrt{\frac{1}{2} \dot{\varepsilon}' \dot{\varepsilon} }$

Effective isotropic power-law viscosity: $\eta_\text{eff} = \eta_0 \dot{\varepsilon}_{II}^{\frac{1}{n} - 1}$



In [219]:
function Stress_PowerLawViscous(ε̇, η0, n)
    
    ε̇II   = sqrt(1/2*(ε̇[1]^2 + ε̇[2]^2) + ε̇[3]^2)

    η_eff = η0 * ε̇II^(1/n-1)

    return [
        2*η_eff*ε̇[1],
        2*η_eff*ε̇[2],
        2*η_eff*ε̇[3]
    ]
end

Stress_PowerLawViscous (generic function with 1 method)

In [220]:
ε̇ = [-1e-15, 1e-15, 1e-16]
η0, n = 1e12, 3.0
J = Enzyme.jacobian(ReverseWithPrimal, Stress_PowerLawViscous, ε̇, η0, n)
display(J.val)
display(J.derivs[1])

3-element Vector{Float64}:
 -1.993377434954683e7
  1.993377434954683e7
  1.9933774349546828e6

3×3 transpose(::Matrix{Float64}) with eltype Float64:
 1.3355e22   6.5788e21   1.31576e21
 6.5788e21   1.3355e22  -1.31576e21
 6.5788e20  -6.5788e20   1.98022e22

## Elasto-plasticity 2D - incremental form

*Predictor-corrector scheme*

Trial state
$$
\tau = \tau^0 + D \mathbf{\delta \varepsilon}
$$

Corrected state:
$$
\tau = \tau^0 + D \left( \mathbf{\delta \varepsilon - \delta \varepsilon^\textrm{p}} \right)
$$


*Drucker-Prager model*

Yield function: $F = \tau_{II} - C\cos{\phi} - P\sin{\phi}$

Flow potential: $Q = \tau_{II} - C\cos{\psi} - P\sin{\psi}$

Plastic strain increment:  $\delta \varepsilon^\textrm{p} = \lambda \frac{\partial Q}{\partial \mathbf{\tau}}$ 

In [221]:
function Stress_ElastoPlastic( Δε, τ0, K, G, C, ϕ, ψ)

    # Plastic mutilpler
    λ = 0.0

    # Elastic trial state
    τ = [ 
        τ0[1] + 2*G*Δε[1],
        τ0[2] + 2*G*Δε[2],
        τ0[3] + 2*G*Δε[3],
        τ0[4] -   K*Δε[4]
    ]

    # Yield function
    τII = sqrt(1/2*(τ[1]^2 + τ[2]^2) + τ[3]^2)
    P   = τ[4]
    F   = τII - C*cosd(ϕ) - P*sind(ϕ)

    if F>0
        λ = F/(G + K*sind(ψ)*sind(ϕ)) 
    end

     # Plastic strains
    Δεp = [ 
        λ * τ[1]/2/τII,
        λ * τ[2]/2/τII,
        λ * τ[3]/1/τII,
        λ * sind(ψ),
    ]

    # Corrected stress
    τ = [ 
        τ0[1] + 2*G*(Δε[1] - Δεp[1]  ),
        τ0[2] + 2*G*(Δε[2] - Δεp[2]  ),
        τ0[3] + 2*G*(Δε[3] - Δεp[3]/2),
        τ0[4] -   K*(Δε[4] - Δεp[4]  )
    ]

    τII = sqrt(1/2*(τ[1]^2 + τ[2]^2) + τ[3]^2)
    P   = τ[4]
    Fc   = τII - C*cosd(ϕ) - P*sind(ϕ)

    # @show F, Fc

return τ
end

Stress_ElastoPlastic (generic function with 1 method)

### First, let's try the elastic limit

In [222]:
ε    = [-1e-4, 1e-4, -1e-5, 1e-5]
τ0   = [5e7, -5e7, 0.0, 1e8]
K, G = 1e11, 5e10
C    = 1e7
ϕ, ψ = 35.0, 10.0

J = Enzyme.jacobian(ReverseWithPrimal, Stress_ElastoPlastic, ε, τ0, K, G, C, ϕ, ψ)

display(J.val)

display(J.derivs[1])

4-element Vector{Float64}:
  4.0e7
 -4.0e7
 -1.0000000000000001e6
  9.9e7

4×4 transpose(::Matrix{Float64}) with eltype Float64:
 1.0e11  0.0     0.0      0.0
 0.0     1.0e11  0.0      0.0
 0.0     0.0     1.0e11   0.0
 0.0     0.0     0.0     -1.0e11

### Second, we change the pre-loading to reach the plastic limit

In [223]:
τ0   = [5e7, -5e7, 0.0, 1e6]

J = Enzyme.jacobian(ReverseWithPrimal, Stress_ElastoPlastic, ε, τ0, K, G, C, ϕ, ψ)

display(J.val)

display(J.derivs[1])

4-element Vector{Float64}:
       1.3473139154521637e7
      -1.3473139154521637e7
 -336828.47886304086
       9.21556060353881e6

4×4 transpose(::Matrix{Float64}) with eltype Float64:
 2.51523e10   8.53052e9    4.26526e8   -4.78149e10
 8.53052e9    2.51523e10  -4.26526e8    4.78149e10
 2.13263e8   -2.13263e8    3.36722e10   1.19537e9
 1.44758e10  -1.44758e10  -7.2379e8    -8.33889e10